In [1]:
from electroviz.core.experiment import Experiment

exp = Experiment("E:/RG007/")

Parsing directories...
Loading NIDAQ data...
Loading Imec data...
Loading Kilosort data...


C:\Ryan\GitHub\electroviz\electroviz\streams\spikes.py:109: RuntimeWarning: Mean of empty slice
  cluster_depths.append(np.nanmean(spike_depths[cluster_idx]))


Aligning Syncs...
Loading bTsS data...
Creating Stimuli...
Creating Populations...
Creating Probes...
Done!
